Поиск минимального значения **log_loss** (и номера соответствующей итерации) для датасета  «Predicting a Biological Response»

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import log_loss

Загружаем данные из файла gbm-data.csv,  оставляем фичи, которые
содержат различные характеристики молекулы.
Также разбиваем выборку на обучающую и тестовую. 

In [0]:
df = pd.read_csv('gbm-data.csv')
y = df['Activity'].values
X = df.loc[:, 'D1':'D1776'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=241)

Опредялем сигмоидную функцию (для получения вероятностей)



In [0]:
def sigmoid(y_pred):
    return 1.0 / (1.0 + math.exp(-y_pred))

Определяем функцию **log-loss**. Для этого используем метод **staged_decision_function** для предсказания качества на обучающей и тестовой выборкена каждой итерации.

In [0]:
def log_loss(model, X, y):
    results = []
    for pred in model.staged_decision_function(X):
        results.append(log_loss(y, [sigmoid(y_pred) for y_pred in pred]))
    return results

Определяем функцию для построения график значений **log-loss** на обучающей и тестовой выборках. Также добавим поиск минимального значения метрики и номера итерации, на которой оно достигается.

In [0]:
def plot_loss(learning_rate, test_loss, train_loss):
    plt.figure()
    plt.plot(test_loss, 'r')
    plt.plot(train_loss, 'g')
    plt.legend(['test', 'train'])

    min_loss_value = min(test_loss)
    min_loss_index = test_loss.index(min_loss_value)
    return min_loss_value, min_loss_index

Построим модель на основе **GradientBoostingClassifier** для каждого значения **learning_rate** из списка [1, 0.5, 0.3, 0.2, 0.1]

In [0]:
def model_test(learning_rate):
    model = GradientBoostingClassifier(learning_rate=learning_rate, n_estimators=250, random_state=241)
    model.fit(X_train, y_train)

    train_loss = log_loss_results(model, X_train, y_train)
    test_loss = log_loss_results(model, X_test, y_test)
    return plot_loss(learning_rate, test_loss, train_loss)

In [0]:
min_loss_results = {}
for learning_rate in [1, 0.5, 0.3, 0.2, 0.1]:
    min_loss_results[learning_rate] = model_test(learning_rate)